In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt


def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        if os.path.isfile(image_path):
            image = Image.open(image_path).convert('RGB')
            images.append((image, filename))
    return images


transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),  
    transforms.ToTensor()
])


class ControlNet(torch.nn.Module):
    def __init__(self):
        super(ControlNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.resnet(x)).squeeze()


def apply_control_net(image_tensor, control_net):
    control_signal = control_net(image_tensor)
    augmented_image = (image_tensor * control_signal * 255).clamp(0, 255).byte()
    return augmented_image


input_folder = r"C:\Users\MY\Desktop\infrastructure"
images = load_images_from_folder(input_folder)


control_net = ControlNet()


output_folder = r"C:\Users\MY\Desktop\aug_infrastructure"
os.makedirs(output_folder, exist_ok=True)

for image, filename in images:
    
    for i in range(5):  
        image_tensor = transform(image).unsqueeze(0)
        augmented_image_tensor = apply_control_net(image_tensor, control_net)
        augmented_image = augmented_image_tensor.squeeze(0).permute(1, 2, 0).numpy().astype(np.uint8)
        output_image_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_{i}.jpg')
        augmented_image_pil = Image.fromarray(augmented_image)
        augmented_image_pil.save(output_image_path)

print("Augmentation process is complete. Augmented images saved in:", output_folder)
